## STEP 1: Understand the Table Structure

Read Data from Bronze layer

In [0]:
df_bronze = spark.read.table("dev_project.bronze.crm_prd_info")
display(df_bronze.limit(10))
#check schema
df_bronze.printSchema()




## STEP 2: Get a Row Count & Sample Data

**What to observe:**
- Is the row count what you expect?
- Do the columns look reasonable?
- Are there obvious issues (extra spaces, weird formatting)?

In [0]:
#Get basic stats
#df_bronze.describe().show()
print(f"Total records: {df_bronze.count()}")
df_bronze.describe().show()

In [0]:
%sql
select * from dev_project.bronze.crm_prd_info limit 10;

## STEP 3: Analyze NULL/Missing Values


Trimming

In [0]:
from pyspark.sql.functions import col, trim
from pyspark.sql.types import StringType

df_bronze = df_bronze.select([
    trim(col(c)).alias(c) if isinstance(t, StringType) else col(c)
    for c, t in df_bronze.dtypes
])
display(df_bronze.limit(10))

In [0]:
from pyspark.sql import functions as F

# Parse product key: extract category and update prd_key
if "prd_key" in df_bronze.columns:
    df_bronze = df_bronze.withColumn(
        "cat_id",
        F.regexp_replace(F.substring(F.col("prd_key"), 1, 5), "-", "_")
    )
    df_bronze = df_bronze.withColumn(
        "prd_key",
        F.substring(F.col("prd_key"), 7, F.length(F.col("prd_key")))
    )

display(df_bronze.limit(10))

This code replaces all NULL values in the "prd_cost" column with 0, while keeping all existing non-null values unchanged.

In [0]:
df_bronze = df_bronze.withColumn("prd_cost", F.coalesce(col("prd_cost"), F.lit(0)))
display(df_bronze.limit(10))

Replace with a Default Date to  indicate "no end date" or "currently active" records.

In [0]:
from pyspark.sql.functions import coalesce, col, lit

# Replace null end dates with '9999-12-31' to indicate "active/ongoing"
df_bronze = df_bronze.withColumn("prd_end_dt", 
    F.coalesce(col("prd_end_dt"), F.lit("9999-12-31"))
)
display(df_bronze.limit(10))

Product Line Normalization

In [0]:
df_bronze = (
    df_bronze
    # Normalize product line
    .withColumn(
        "prd_line",
        F.when(F.upper(col("prd_line")) == "M", "Mountain")
         .when(F.upper(col("prd_line")) == "R", "Road")
         .when(F.upper(col("prd_line")) == "S", "Other Sales")
         .when(F.upper(col("prd_line")) == "T", "Touring")
         .otherwise("n/a")
    )
)
display(df_bronze.limit(10))

Date Casting:  
The Problem:
When data is loaded into a PySpark DataFrame (especially from CSV, JSON, or text files), date columns are often read as strings, not actual date objects.

In [0]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import col, coalesce, lit, datediff

df_bronze = df_bronze.withColumn("prd_start_dt", col("prd_start_dt").cast(DateType()))
df_bronze = df_bronze.withColumn("prd_end_dt", col("prd_end_dt").cast(DateType()))

df_bronze = df_bronze.withColumn("prd_end_dt", 
    coalesce(col("prd_end_dt"), lit("9999-12-31").cast(DateType()))
)

df_bronze = df_bronze.withColumn("product_lifespan_days", 
    datediff(col("prd_end_dt"), col("prd_start_dt"))
)
display(df_bronze.limit(10))


Renaming Columns

In [0]:
RENAME_MAP = {
    "prd_id": "product_id",
    "cat_id": "category_id",
    "prd_key": "product_number",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date"
}
for old_name, new_name in RENAME_MAP.items():
    df_bronze = df_bronze.withColumnRenamed(old_name, new_name)
df_bronze.limit(10).display()

In [0]:
df_bronze.write.mode("overwrite").format("delta").saveAsTable("dev_project.silver.crm_products")

Sanity checks of customer info table

In [0]:

%sql
SELECT * FROM dev_project.silver.crm_products LIMIT 10